In [1]:
import os
import re

import warnings

warnings.simplefilter("ignore", UserWarning)

from matplotlib import pyplot as plt
%matplotlib inline

import pandas as pd

pd.options.mode.chained_assignment = None

import numpy as np
from string import punctuation

from nltk.tokenize import word_tokenize

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, auc, roc_auc_score
from sklearn.externals import joblib

import scipy
from scipy.sparse import hstack

In [2]:
df_dataset = pd.read_csv('./dataset_inspur.csv', encoding='utf-8', usecols=['COMMCONTENT_SEG', 'COMMLEVEL'])
df_dataset.columns = ['COMMCONTENT_SEG', 'COMMLEVEL']

In [3]:
df_dataset

,COMMCONTENT_SEG,COMMLEVEL
0,普通 公园 一个 只是 多 了 几个 泉 而已 人不多 适合 老人 孩子 闲逛 买票 的话 ...,1
1,跟 儿子 在 里面 玩 了 一天 非常 好 跟 儿子 在 里面 玩 了 一天 非常 好 真的...,1
2,这 已经 是 第五次 来 这里 玩 了 每次 孩子 都 很 喜欢 不 愿意 从水里 出来 有...,1
3,当天 在 携程 上定 的 票 打 温泉 度假村 咨询电话 和 携程 客服 都 说 次日 生效...,1
4,烟台 历史 的 一部分 非常 值得 推荐 去 看看 海边 景色 也 很漂亮,1
5,周末 看看 动物 亲近 亲近 大自然 挺 好 的 媳妇儿 还 跟 猴子 拍照 猴子 满身 爬...,1
6,五四 广场 青岛 旅游景点 必 打卡 又 一 地点 标志性 红包 建筑 雕塑 矗立 在 市政...,1
7,五四 广场 坐落 在 山东省 青岛市 市南区 在 海边 那个 红红的 火炬 就是 五四 广场...,1
8,环境 好 景色 美 值得 游览 感觉 很 好,1
9,青岛市 中心广场 纪念 五四运动 而 命名 风景 独特,1


In [7]:
data = df_dataset.sample(frac=1, random_state=42)
print(data.shape)

(19994, 2)


In [8]:
data.head(10)

,COMMCONTENT_SEG,COMMLEVEL
7992,商业 气息 太 浓 摆摊 的 太 多 破坏 了 村子 的 整体 感觉,2
5217,刘公岛 是 威海 最 值得 来 的 景点 做好 能 避开 周末 游客 多 的 时间 有 时间...,1
16507,泰山 温泉 周围环境 景色 很 美 室内 小 池子 温泉水 不 干净 泳池 水太凉 一般,3
9121,德国 风情街 在 馆陶 路 那边 有 一些 德国 风情 的 建筑 适合 拍照 哦,2
3241,值得 去 看 一下 啊 这里 也 是 一个 风景区 啊,1
2980,总的来说 不错 和 朋友 一起 去 的 就是 人太多 了 不能 安静 享受 在 观唐 的 时...,1
9801,感觉 没有 传说 中 那么 可怕 不过 楼梯 数量 相当可观 歇 了 n 次 才 到 南天门...,2
15861,下午 我们 到 到 蓬莱阁 游玩 蓬莱阁 景区 至少 有 三扇 门 网络 票 必须 南门 进...,3
18130,过于 商业化 感觉 花钱 买票 进入 了 一个 小商品 市场,3
16288,天气 热 每个 项目 都 要 排队 等 好久 老人 热 的 不行 孩子 也 不愿 等 花 了...,3


In [11]:
for row in data.head(10).iterrows():
    print(row[1]['COMMLEVEL'], row[1]['COMMCONTENT_SEG']) 

2 商业 气息 太 浓 摆摊 的 太 多 破坏 了 村子 的 整体 感觉
1 刘公岛 是 威海 最 值得 来 的 景点 做好 能 避开 周末 游客 多 的 时间 有 时间 可以 细细 玩 除了 动物园 比较 鸡肋 其他 景点 还好 喜欢 坐船 的 可以 座 环岛 的 船
3 泰山 温泉 周围环境 景色 很 美 室内 小 池子 温泉水 不 干净 泳池 水太凉 一般
2 德国 风情街 在 馆陶 路 那边 有 一些 德国 风情 的 建筑 适合 拍照 哦
1 值得 去 看 一下 啊 这里 也 是 一个 风景区 啊
1 总的来说 不错 和 朋友 一起 去 的 就是 人太多 了 不能 安静 享受 在 观唐 的 时间 很 赶 的 感觉 看到 好多 带 宝宝 去 的 天 太冷 注意 保暖 啊 小孩 似乎 受不了 温差 的 折腾 自助餐 看起来 品种 很多 其实 吃 到 的 不太多 菜 颜色 不 鲜艳 没什么 食欲 饮料 难 喝 就 细节 来说 观唐 服务 还是 很 贴心 的 啊 滑梯 楼梯 前 总有 一位 工作人员 提醒 大家 注意安全 很 贴心 有 机会 温差 不 大 带 孩子 再 去
2 感觉 没有 传说 中 那么 可怕 不过 楼梯 数量 相当可观 歇 了 n 次 才 到 南天门 路上 有 很多 挑山工 看着 他们 再 看 山上 的 物价 也 就 觉得 不是 那么 贵 了
3 下午 我们 到 到 蓬莱阁 游玩 蓬莱阁 景区 至少 有 三扇 门 网络 票 必须 南门 进 老婆 在 携程 预付 的 票 原价 140 元 的 门票 网络 价 125 元 预付 再 返现 5 元 在 蓬莱 市区 有 卖 125 元 的 蓬莱阁 门票 也 有 卖 120 元 的 其实 都 一样 没 必要 网上 订 蓬莱阁 景区 很大 怕累 的 同志 可以 坐 电瓶车 10 元 位 其实 125 元 的 门票 只是 蓬莱阁 古建筑群 用 外面 的 水师 衙门 登州 博物馆 田 横山 等 都 不用 门票 有人 问 就 说 昨天 没看 完 今天 来看 门票 没带 不过 看 蓬莱阁 的 趋势 以后 会 出个 大 门票 把 所有 的 景点 全部 包含 在内 蜂友们 就 没得 逃票 了 现在 其实 也 都 包含 在 高高的 城墙 内 了 只是 大门口 没 人 检票 而已 古建筑群 入口 现在 有人 把守 严格 查票 

### 训练集/测试集 划分

In [12]:
x_train, x_test, y_train, y_test = train_test_split(data['COMMCONTENT_SEG'],
                  data['COMMLEVEL'],
                  test_size=0.2,
                   random_state=42,
                   stratify=data['COMMLEVEL'])

print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(15995,) (3999,) (15995,) (3999,)


#### 将测试标签保存在磁盘上供以后使用

In [ ]:
pd.DataFrame(y_test).to_csv('./predictions/y_true.csv', index=False, encoding='utf-8')

In [20]:
n_classes = np.unique(y_train).shape[0]

In [21]:
n_classes

3

In [41]:
x_train[x_train.isnull()==True]

Series([], Name: COMMCONTENT_SEG, dtype: object)

In [40]:
x_train[14365] = "nooo"

## 1 - 基于词ngrams的词袋模型

In [55]:
vectorizer_word = TfidfVectorizer(max_features=40000,
                                  min_df=5,
                                  max_df=0.5,
                                  analyzer='word',
                                  stop_words='english',
                                  ngram_range=(1, 2))

vectorizer_word.fit(x_train)

tfidf_matrix_word_train = vectorizer_word.transform(x_train)
tfidf_matrix_word_test = vectorizer_word.transform(x_test)

In [56]:
tfidf_matrix_word_train

<15995x9696 sparse matrix of type '<class 'numpy.float64'>'
	with 254816 stored elements in Compressed Sparse Row format>

In [52]:
# 
lr_word = LogisticRegression(solver='sag', 
                             verbose=2,
                            random_state=42)
lr_word.fit(tfidf_matrix_word_train, y_train)

convergence after 20 epochs took 1 seconds
convergence after 24 epochs took 0 seconds


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s


convergence after 21 epochs took 0 seconds


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.5s finished


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=42, solver='sag', tol=0.0001,
          verbose=2, warm_start=False)

In [53]:
joblib.dump(lr_word, './lr_word_ngram.pkl')

y_pred_word = lr_word.predict(tfidf_matrix_word_test)
pd.DataFrame(y_pred_word, columns=['y_pred']).to_csv('./lr_word_ngram.csv', 
index=False)

In [54]:
y_pred_word = pd.read_csv('./lr_word_ngram.csv')
print(accuracy_score(y_test, y_pred_word))

0.663915978995


In [63]:
!echo 0.66416604151 >> word-ngram-bag-of-words-0.66.txt

In [59]:
vectorizer_char = TfidfVectorizer(max_features=40000,
                                  min_df=5,
                                  max_df=0.5,
                                  analyzer='char',
                                  ngram_range=(1, 4))

vectorizer_char.fit(x_train);

tfidf_matrix_char_train = vectorizer_char.transform(x_train)
tfidf_matrix_char_test = vectorizer_char.transform(x_test)

lr_char = LogisticRegression(solver='sag', verbose=2)
lr_char.fit(tfidf_matrix_char_train, y_train)

y_pred_char = lr_char.predict(tfidf_matrix_char_test)
joblib.dump(lr_char, './lr_char_ngram.pkl')

pd.DataFrame(y_pred_char, columns=['y_pred']).to_csv('./lr_char_ngram.csv', 
index=False)

y_pred_char = pd.read_csv('./lr_char_ngram.csv')
print(accuracy_score(y_test, y_pred_char))

convergence after 20 epochs took 0 seconds


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.7s remaining:    0.0s


convergence after 20 epochs took 1 seconds
convergence after 21 epochs took 1 seconds
0.678419604901


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    2.0s finished


In [60]:
print(accuracy_score(y_test, y_pred_char))

0.678419604901


In [64]:
!echo 0.678419604901 >> char-ngram-bag-of-words-0.67.txt